In [3]:
import flask
from flask import Flask,jsonify,render_template
import pickle
from sklearn.model_selection import train_test_split
from google.colab import files
Heart_Disease = files.upload()
!unzip Heart_Disease.zip

Saving Heart_Disease.zip to Heart_Disease.zip
Archive:  Heart_Disease.zip
   creating: model/
  inflating: model/heart_model.pkl   
   creating: templates/
  inflating: templates/index.html    
  inflating: templates/main.html     
  inflating: app.ipynb               
  inflating: cleveland.csv           
  inflating: Heart_project_IEEE.ipynb  


In [0]:
import numpy as np
import pandas as pd
df = pd.read_csv('cleveland.csv', header = None)
df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']
df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


###Install ngrok first if you haven't already (pip code is commented)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
with open(f'model/heart_model.pkl', 'rb') as f:
    model = pickle.load(f)
app = Flask(__name__, template_folder='templates')

def input_scaler(age,sex,cp,trestbps,chol,fb,restecg,thalach,exang,oldpeak,slope,ca,thal):
  if sex == 'female': sex = 0
  else : sex = 1
  X_train, _, _, _ = train_test_split(X, y, test_size = 0.18,random_state = 42)
  input_np = [age,sex,cp,trestbps,chol,fb,restecg,thalach,exang,oldpeak,slope,ca,thal]
  X_train = np.vstack([X_train,input_np])
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train)
  return X_train[-1:]
 
# INSTALL NGROK if error occurs

#!pip install flask==0.12.2
#!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    age = flask.request.form['age']
    sex = flask.request.form['sex']
    cp = flask.request.form['cp']
    trestbps = flask.request.form['trestbps']
    chol = flask.request.form['chol']
    fbs = flask.request.form['fbs']
    restecg = flask.request.form['restecg']
    thalach = flask.request.form['thalach']
    exang = flask.request.form['exang']
    oldpeak = flask.request.form['oldpeak']
    slope = flask.request.form['slope']
    ca = flask.request.form['ca']
    thal = flask.request.form['thal']
 
    

    input_scaled = input_scaler(age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal)
    prediction = model.predict(input_scaled)
    output = prediction[0]

    if (output == 1): 
        a = "You should see a heart doctor as soon as possible. Take Care"
        return render_template('main.html', Result =' {}'.format(a))
    else:
        a = "Don't worry, you are safe. Take Care"
        return render_template('main.html', Result =' {}'.format(a))

if __name__ == '__main__':
    app.run()
    



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eb67244f90d7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2020 22:43:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 22:43:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2020 22:44:13] "POST /predict HTTP/1.1" 200 -
